In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime
from keras.optimizers import SGD
from tensorflow.keras import datasets, layers, models, regularizers
import matplotlib.pyplot as plt

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


ImportError: cannot import name 'SGD' from 'keras.optimizers' (C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizers.py)

In [ ]:
## Set flags

# Set value to True to upload results to TensorBoard
TENSORBOARD = False
# Set value to True to have the neural network save its weights
SAVEMODE = False
# Set value to the path to the directory where the neural network will load its weights
# if not loading set to None
LOADDIR = None

In [ ]:
def preprocessDataset(dataset):
    """
    Preprocesses an entire set of datapoints so that they are usable by a neural network

    :param dataset: The dataset that is to be preprocessed in the form (images, labels)
    :return: The preprocessed dataset in the form (processedImages, processedLabels)
    """

    # Unpacks the dataset into the images (input data) and labels (expected output data)
    images, labels = dataset

    # Preprocesses the input data
    newImages = images.reshape(images.shape[0], images.shape[1], images.shape[2], 3)
    # Condenses the values of the images so that they fall within 0 and 1
    newImages = newImages / 255.0

    # Preprocesses the expected output data

    # Collapses the n-dimensional array into a 1D array
    # For example:
    # [ [ 1, 2 ], [ 3, 4 ] ]
    # [ 1, 2, 3, 4 ]
    newLabels = labels.flatten()

    # Converts the labels from a 1D array of numbers ranging from 1 - 10 to a 2D array of 0s
    # in which each number in the 1D array has a respective array within the 2D array, and
    # the value of said number is the index in the other array that is a 1
    #
    # For example:
    # 1D array: [ 1, 2, 2, 4, 3 ]
    # One Hotted 2D Array:
    # [ [ 1, 0, 0, 0 ],   // 1
    #   [ 0, 1, 0, 0 ],   // 2
    #   [ 0, 1, 0, 0 ],   // 2
    #   [ 0, 0, 0, 1 ],   // 4
    #   [ 0, 0, 1, 0 ] ]  // 3
    newLabels = tf.one_hot(newLabels.astype(np.int32), depth=10)

    return newImages, newLabels


def getRawDatasets():
    """
    :return: The unprocessed datasets that the neural network is to be trained on and tested against
    """

    # Using the CIFAR-10 dataset
    return tf.keras.datasets.cifar10.load_data()

In [ ]:
def getTrainingConfigurations():
    """
    Epochs: The amount of times the neural network trains against the entire training dataset
    Batch Size: The amount of samples ran through before the gradient update is applied
    Callbacks: The set of callbacks that will be applied during the training (e.g. saving)
    Optimizer: The method that will be used in training the neural network
    Loss Function: The loss function the neural network will use to judge its performance
    Metrics: The metrics that will allow the user to understand the performance of the neural network

    :param saveDir: The directory where the neural network is saved, indicated only if loading a neural network
    :return: A tuple of the configurations described above, in order
    """

    epochs = 300
    batchSize = 32

    # Gets the datetime now as to create unique folders
    timeNow = datetime.now().strftime("%Y%m%d-%H%M%S")
    # The folder the fitness logs will be saved to
    logDir = "logs/fit/" + timeNow
    # The directory the neural network will be saved to
    saveDir = "save_files/" + timeNow

    callbacks = [tf.keras.callbacks.TensorBoard(log_dir=logDir, histogram_freq=1)]

    if SAVEMODE:
        callbacks.append(tf.keras.callbacks.ModelCheckpoint(filepath=saveDir, verbose=1))

    # Using Stochastic Gradient Descent, which estimates the gradient of the entire dataset
    # instead of calculating the gradient at the current data point
    optimizer = tf.keras.optimizers.SGD(lr=0.001, momentum=0.9)

    # Calculates the loss based on the difference in value of the expected and actual outputs, with
    # respect to the other outputs
    loss = "categorical_crossentropy"

    # Accuracy is being used as a user-friendly way of displaying the effectiveness of the neural network
    metrics = ["accuracy"]

    return epochs, batchSize, callbacks, optimizer, loss, metrics

In [ ]:
def createModel(train_images):
  """
  Creates and returns the model of our CNN
  Num_classes: The number of classes (categories) in the CIFAR-10 dataset
  Model: The model of our CNN to be returned

  #TODO: Make this more elegant??
  :param: Set of training images, used to determine the input shape
  :return: An object of type tf.keras.Model
  """
  # 10 categories in the CIFAR-10 dataset
  num_classes = 10

  model = models.Sequential()
  model.add(layers.Conv2D(32, 3, padding='same', input_shape=train_images.shape[1:], activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(32, 3, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))

  model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(64, 3, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.45)) # was 0.35

  model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(128, 3, activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.5)) # was 0.45

  model.add(layers.Flatten())
  model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.L2(0.001))) # added new regularizer
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes, activation='softmax'))

  model.summary()

  return model

In [ ]:
# Main function
if __name__ == "__main__":
    # Training configurations
    epochs, batchSize, callbacks, optimizer, loss, metrics = getTrainingConfigurations()

    # Raw datasets
    rawTrainingDataset, rawTestingDataset = getRawDatasets()

    # Preprocessed datasets
    trainingImages, trainingLabels = preprocessDataset(rawTrainingDataset)
    testingImages, testingLabels = preprocessDataset(rawTestingDataset)

    # Model
    model = createModel(trainingImages)

    # Compile model, uses cross entropy as a loss function
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    # Set up TensorBoard
    log_dir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")

    if LOADDIR is not None:
        model.load_weights(LOADDIR)
    
    # Begin training the model
    history = model.fit(trainingImages, trainingLabels, epochs=epochs, validation_data=(testingImages, testingLabels), batch_size=batchSize, callbacks=callbacks)

    # TensorBoard upload
    if TENSORBOARD:
      !tensorboard dev upload --logdir ./logs \
        --name "INT2 Group 28 CIFAR-10" \
        --description "Training results from https://github.com/UOY-Int2-Group-28/Final-Project" \
        --one_shot

    # Create plot
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.ylim([0.5, 1])
    plt.legend(loc='lower right')

    test_loss, test_acc = model.evaluate(testingImages, testingLabels, verbose=2)